## Post Call Analytics (PCA) Using Amazon Bedrock
Amazon Bedrock을 사용한 통화 후 분석 사용 사례에 대한 이 교육에 오신 것을 환영합니다.

기업이 다양한 채널을 통해 고객과 계속 상호 작용함에 따라 이러한 상호 작용을 분석하여 고객 행동 및 선호도에 대한 인사이트를 얻는 것이 점점 더 중요해지고 있습니다.<BR>
통화 후 분석은 통화 종료 후 고객 상호 작용을 분석하는 방법 중 하나입니다.<BR>
대규모 언어 모델을 사용하면 보다 정확한 감정 분석을 가능하게 하고, 특정 고객의 요구와 선호도를 파악하며, 전반적인 고객 경험을 개선함으로써 통화 후 분석의 효과를 크게 높일 수 있습니다.<BR>

이 샘플 노트북에서는 통화 후 분석에 Bedrock을 사용할 때 얻을 수 있는 다양한 이점과 기업이 현대 시장에서 경쟁 우위를 확보할 수 있는 방법을 다음 주제를 통해 살펴봅니다.<BR>

- 베드락에서 LLM 모델 선택(타이탄 텍스트 및 앤트로닉 클로드)
- 하나의 모델로 여러 PCA 작업 처리
- 긴 통화 기록 처리

# Environment Setup
샘플 코드를 실행하는 데 필요한 패키지를 설치 및 업그레이드합니다. <BR>
**참고: 업데이트된 패키지를 사용하려면 커널을 재시작해야 할 수 있습니다.**

# Auto Reload

In [2]:
%load_ext autoreload
%autoreload 2

# Import packages

In [6]:
import os
import sys

module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [7]:
import json
import time
import boto3
import langchain
from time import strftime
from termcolor import colored
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock
from utils import bedrock
from utils.s3 import s3_handler

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Speech To Text (STT)

## Upload data to s3

In [148]:
s3 = s3_handler()

This is a S3 handler with [None] region.


In [149]:
prefix = "dj"#<your-name>
bucket_name = f'bedrock-training-{prefix}'
data_dir = "./records"

In [150]:
s3.create_bucket(bucket_name)
source_dir, target_bucket, target_dir = data_dir, bucket_name, "/records"
s3.upload_dir(source_dir, target_bucket, target_dir)
data_path_s3 =f's3://{bucket_name}/records/voice-examples.wav'

CREATE:[bedrock-training-dj] Bucket was created successfully
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Upload:[./records] was uploaded to [s3://bedrock-training-dj/records]successfully


## Transcribe

In [151]:
transcribe_client = boto3.client('transcribe')

In [152]:
create_date = strftime("%m%d-%H%M%s")
job_name = f'{prefix}-stt-job-{create_date}'
print (f's3 data path: {data_path_s3}')
print (f'job_name: {job_name}')

s3 data path: s3://bedrock-training-dj/records/voice-examples.wav
job_name: dj-stt-job-0912-08471694508478


### Run Transcribe Job

In [153]:
transcribe_client.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': data_path_s3},
    MediaFormat='wav',
    Settings={
        'ShowSpeakerLabels': True,
        'MaxSpeakerLabels': 2,
    },
    LanguageCode='ko-KR'
)

{'TranscriptionJob': {'TranscriptionJobName': 'dj-stt-job-0912-08471694508478',
  'TranscriptionJobStatus': 'IN_PROGRESS',
  'LanguageCode': 'ko-KR',
  'MediaFormat': 'wav',
  'Media': {'MediaFileUri': 's3://bedrock-training-dj/records/voice-examples.wav'},
  'StartTime': datetime.datetime(2023, 9, 12, 8, 47, 58, 882000, tzinfo=tzlocal()),
  'CreationTime': datetime.datetime(2023, 9, 12, 8, 47, 58, 861000, tzinfo=tzlocal()),
  'Settings': {'ShowSpeakerLabels': True, 'MaxSpeakerLabels': 2}},
 'ResponseMetadata': {'RequestId': 'af78adf3-97bc-4f25-9c3c-00674d103162',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'af78adf3-97bc-4f25-9c3c-00674d103162',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '359',
   'date': 'Tue, 12 Sep 2023 08:47:58 GMT'},
  'RetryAttempts': 0}}

### Check Job Status

In [154]:
while True:
    status = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(10)
print(status)

Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
{'TranscriptionJob': {'TranscriptionJobName': 'dj-stt-job-0912-08471694508478', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'ko-KR', 'MediaSampleRateHertz': 8000, 'MediaFormat': 'wav', 'Media': {'MediaFileUri': 's3://bedrock-training-dj/records/voice-examples.wav'}, 'Transcript': {'TranscriptFileUri': 'https://s3.us-east-1.amazonaws.com/aws-transcribe-us-east-1-prod/738515539849/dj-stt-job-0912-08471694508478/1bb06137-eb89-4c51-95bd-4617441cf798/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjECgaCXVzLWVhc3QtMSJGMEQCIA0spHPT7FXfJKkMURsb0qCnInjChSiuDAH1CzSXQJFxAiBiXf5h4fgbtKbjLTQG222uVYEciRhUq2Mtws5dvyUlqiqyBQgREAQaDDI3NjY1NjQzMzE1MyIMSdMVrQO7vG0pdgU8Ko8F%2BhVWNb5NwA7A9jGluZ9HATf2EwZ9gUYe24F79M4LR7o8SNumgO78BF7l0m1CxW%2B2cRq%2BhXKbiULztNifa1w%2BeuX9yrn%2F

### Retrieve Text

In [156]:
def spk_sperator(data):
    
    previos_spk = ""
    contents, contents_temp = [], []
    
    for res in data["results"]["items"]:

        speaker_label = res["speaker_label"]
        content = res["alternatives"][0]["content"]

        if previos_spk != speaker_label:
            contents.append(" ".join(contents_temp))
            contents_temp = []
            contents_temp.append(f'{speaker_label}:')
            contents_temp.append(content)
        else:
            contents_temp.append(content)
            
        previos_spk = speaker_label

    contents.append(" ".join(contents_temp))
    
    return "\n".join(contents[1:])
    
if status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    response = urlopen(status['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    data = json.loads(response.read())
    text = data['results']['transcripts'][0]['transcript']
    #print(text)
    
    text = spk_sperator(data)
    print (text)

spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
spk_0: 그렇습니다 .
spk_1: 그러면 이거 어떻게 처리해 줄 거예

# Post Call Analysis

## Choice of models in Bedrock
Choose FMs from Amazon, AI21 Labs and Anthropic to find the right FM for your use case.

**Select nternal-use: True(aws 직원), False (고객)** <BR>
**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [157]:
is_internal_use = True #<internal-use> ## if 고객: False, aws직원: True

In [158]:
bedrock_region = "us-east-1" #<your region> ## "us-east-1" or "us-west-2"

In [159]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [160]:
if is_internal_use:
    boto3_bedrock = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    boto3_bedrock = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [161]:
bedrock_models = {
    "Claude" : "anthropic.claude-v1",
    "TitanText": "amazon.titan-tg1-large", 
    "Claude-instant":"anthropic.claude-instant-v1",
    "Claude-V2" : "anthropic.claude-v2",
}

max_tokens = {
    "Claude" : 12000,
    "TitanText": 4096,
    "Claude-instant": 9000,
    "Claude-V2" : 12000,
}

max_tokens = {"Claude" : 120, "TitanText": 130, "Claude-instant": 120, "Claude-V2" : 120}


In [162]:
# Choose one of the bedrock model
model = "Claude-V2" # "Claude", "TitanText", "Claude-instant"
if model in ["Claude", "Claude-instant", "Claude-V2"]:
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=boto3_bedrock,
        model_kwargs={
            "max_tokens_to_sample":512,
            "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원", "\n\n"],
            "temperature":0,
            "top_p":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )
elif model == "TitanText":
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )

## Prompt Template
이 노트북에서는 네 가지 분석(**요약, 감성, 의도, 해결**)을 수행하게 되며, 각 분석에 대한 템플릿이 필요합니다.

In [167]:
summary_template_ko = """
아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "{transcript}"

요약:"""

question_template_ko = """
아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "{transcript}"

질문: "{question}"

응답: """

## Generate Analysis

In [179]:
def analysis(llm, transcript, params, question=None, template="", max_tokens=50):
 
    if question is None:
        prompt = PromptTemplate(template=template, input_variables=["transcript"])
        analysis_prompt = prompt.format(transcript=transcript)
    else:
        prompt = PromptTemplate(template=template, input_variables=["transcript", "question"])
        analysis_prompt = prompt.format(
            transcript=transcript,
            question=question
        )
        
    llm.model_kwargs = params
        
    print (colored(analysis_prompt, 'green'))
    response = llm(analysis_prompt)
    
    return response

In [183]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담사", "\n\n\n", "\n\n질문"],
    "temperature":0,
    "top_p":0.9
}

### Summary

In [181]:
%%time

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    template=summary_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해

### Question and Answer

In [187]:
%%time
question = "고객의 감정은 어떤가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
sp

In [188]:
%%time
question = "문제에 대한 개선을 위해서 어떤 방법이 있을까요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
sp

In [184]:
%%time
question = "학습지는 언제 종료되나요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
sp

In [189]:
%%time
question = "학습지는 언제 종료되고 환불은 언제 가능한가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
sp

In [190]:
%%time
question = "고객이 예전에 살던 곳은 어디인가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
sp

In [191]:
%%time
question = "상담원의 이름은 무엇인가요?"

res = analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    question=question,
    template=question_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 바탕으로 질문에 답하세요

통화: "spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 저희가 하지도 않은 과목을 등록 시켜서 그게 두 달 분이나 결제가 됐더라고요
spk_0: 그러셨군요
spk_1: 저희가 알았으니까 당정 이 이제 올랐으면 그렇게 남의 계좌에서 돈 계속 빼 나갔을 거 아니에요 ? 통제 같은 것도 안 해주고
spk_0: 죄송합니다
spk_1: 그때는 이제 애가 여기 선생님도 좋아하고 학습지도 마음에 들어서 그냥 넘겼어 거든요 남편은 이런 거 그냥 넘기면 안 된다고 했지만 애가 잘 하고 있는데 괜히 선생님 불편하게 만들어서 좋을 거 없으니까요 . 근데 저희 분명히 그만 뒀는데 결제되는 건 좀 아니잖아요 . 한 번 이해하고 넘어갔으면 좀 더 조심해야 조심해 줘야 하는 거 아닌가요 ?
sp

## Handling long call transcripts
LLM의 인풋 토큰 한도를 초과하는 긴 문서를 처리하는 방법을 다룹니다. 

* Map Reduce
![nn](../imgs/map_reduce.png)
출처: https://brain.d.foundation/Engineering/AI/Workaround+with+OpenAI's+token+limit+with+Langchain

* Refine
![nn](../imgs/refine.png)
출처: https://brain.d.foundation/Engineering/AI/Workaround+with+OpenAI's+token+limit+with+Langchain

In [198]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

* prompting to divide and conquer

In [236]:
stuff_prompt_template = """
다음 글을 간단하게 요약해 주세요.

통화: {text}

요약:
"""

chuck_prompt_template = """
다음 글을 간단하게 요약해 주세요.

통화: {text}

요약:
"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """
다음 글을 간단하게 요약해 주세요.

통화: {text}

요약:
"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

* summarize chain

In [237]:
def summary_chain_init(chain_type, llm):


    combine_prompt = PromptTemplate(
        template=combine_prompt_template,
        input_variables=["text"]
    )

    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

In [240]:
def long_call_analysis(llm, transcript, params, chain_type="MAP_REDUCE", max_tokens=50):

    
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning
    
    print (num_tokens, max_tokens)

    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            #separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

        
        summary_chain = summary_chain_init(
            chain_type=chain_type, 
            llm=llm
        )
        response = summary_chain(
            {"input_documents": docs}
        )
        
        print ("Intermediate_steps: \n")
        for idx, step in enumerate(response["intermediate_steps"]):
            print (colored(f'step {idx}: \n', "green"))
            print (colored(f'{step}\n', "green"))
        
        return response["output_text"]
    
    else:
        
        prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])
        analysis_prompt = prompt.format(text=transcript)
        print (colored(analysis_prompt, 'green'))
        
        response = llm(analysis_prompt)
        
        return response
        

In [241]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담사", "\n\n\n", "\n\n질문", "\n\nspk_0", "\n\n통화"],
    "temperature":0,
    "top_p":0.9
}

In [242]:
%%time

res = long_call_analysis(
    llm=llm,
    transcript=text,
    params=PARAMS,
    chain_type="REFINE" # REFINE, MAP_REDUCE
)

print ("Results: \n")
print (res)

3791 50
Now we have 5 documents and the first one has 936 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

다음 글을 간단하게 요약해 주세요.

통화: spk_0: 네 , 고객님 중앙 교육의 이 진역 상담원입니다
spk_1: 안녕하세요 ? 결제 때문에 전화드렸어요 분당 사 때 중앙 교육 학습지 몇 년 했었고요 지금은 학습지 이용 안 하고 있거든요 그리고 저희 서 초로 이사도 했고요 ?
spk_0: 네 , 고객님
spk_1: 근데 며칠 전에 갑자기 여기 학습지 결제 문자가 뜨더라고요 십팔만 원이 결제 됐는데 저희 지금 여기 학습지 이용을 안 하고 있어요 어떻게 된 건가요 ?
spk_0: 알아보겠습니다 .
spk_1: 네 , 빨리 알아봐 주세요 그때 분명히 선생님 하고도 잘 인사 했고 마치고 끝났는데 갑자기 결제 문자가 와서 놀랐거든요 ?
spk_0: 네 , 알아본 결과 학습 종료가 안 된 걸로 나와 있습니다
spk_1: 학습 학습이 종료가 안 됐다고요 ? 저희 저번에 분명히 끝내고 인사까지 했는데요
spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다

요약:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

다음 글을 간단하게 요약해 주세요.

통화: spk_0: 죄송합니다
spk_1: 선생님하고 어떻게 해야해요 ? 길래 학습 종료가 안 돼요 ?
spk_0: 알아보고 처리해 드리겠습니다
spk_1: 처리해 주는 건 당연하고요 그것보다 저희 카드 번호도 분명히 개인 정보인데 해지하면 삭제해야 하는 거 아닌가요 ? 예전에 저희 학습 자 할 때도 결제 금액 이상해서 고객센터에 문의해 보니까 